In [1]:
# imports
import torch
from torchvision import datasets, transforms

In [18]:
# prepare data and normalize
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))]) # change color to greyscale

train = datasets.FashionMNIST('MNIST_data/', download=True, train=True, transform=transform)
train_load = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)

test = datasets.FashionMNIST('MNIST_data/', download=True, train=False, transform=transform)
test_load = torch.utils.data.DataLoader(test, batch_size=64, shuffle=True)

In [20]:
# define neural net
from torch import nn, optim
import torch.nn.functional as F

# nn with all linear layers, Relu activation, dropout regularization, softmax for output
model = nn.Sequential(nn.Linear(784, 256),
                      nn.ReLU(),
                      nn.Dropout(0.2),
                      nn.Linear(256, 128),
                      nn.ReLU(),
                      nn.Dropout(0.2),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Dropout(0.2),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim = 1)
                     )

# loss function, cross entropy
criterion = nn.NLLLoss()

# optimizer function w learning rate
optimizer = optim.Adam(model.parameters(), lr = 0.002)

# run 30 epochs
epochs = 30

train_losses, test_losses = [], []

for e in range(epochs):
  running_loss = 0
  for images, labels in train_load:
    # Flatten Fashion-MNIST images into a 784 long vector
    images = images.view(images.shape[0], -1)
    
    # Training pass
    optimizer.zero_grad()
    
    output = model.forward(images)
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()
    
    running_loss += loss.item()
  else:
    test_loss = 0
    accuracy = 0
    
    # Turn off gradients for validation, saves memory and computation
    with torch.no_grad():
      # Set the model to evaluation mode
      model.eval()
      
      # Validation pass
      for images, labels in test_load:
        images = images.view(images.shape[0], -1)
        log_ps = model(images)
        test_loss += criterion(log_ps, labels)
        
        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim = 1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))
    
    model.train()
    train_losses.append(running_loss/len(train_load))
    test_losses.append(test_loss/len(test_load))
    
    print("Epoch: {}/{}..".format(e+1, epochs),
          "Training loss: {:.3f}..".format(running_loss/len(train_load)),
          "Test loss: {:.3f}..".format(test_loss/len(test_load)),
          "Test Accuracy: {:.3f}".format(accuracy/len(test_load)))

Epoch: 1/30.. Training loss: 0.595.. Test loss: 0.473.. Test Accuracy: 0.828
Epoch: 2/30.. Training loss: 0.451.. Test loss: 0.435.. Test Accuracy: 0.845
Epoch: 3/30.. Training loss: 0.424.. Test loss: 0.401.. Test Accuracy: 0.858
Epoch: 4/30.. Training loss: 0.398.. Test loss: 0.394.. Test Accuracy: 0.859
Epoch: 5/30.. Training loss: 0.384.. Test loss: 0.374.. Test Accuracy: 0.865
Epoch: 6/30.. Training loss: 0.377.. Test loss: 0.365.. Test Accuracy: 0.866
Epoch: 7/30.. Training loss: 0.363.. Test loss: 0.366.. Test Accuracy: 0.867
Epoch: 8/30.. Training loss: 0.352.. Test loss: 0.358.. Test Accuracy: 0.870
Epoch: 9/30.. Training loss: 0.347.. Test loss: 0.377.. Test Accuracy: 0.868
Epoch: 10/30.. Training loss: 0.343.. Test loss: 0.365.. Test Accuracy: 0.875
Epoch: 11/30.. Training loss: 0.340.. Test loss: 0.368.. Test Accuracy: 0.871
Epoch: 12/30.. Training loss: 0.333.. Test loss: 0.353.. Test Accuracy: 0.878
Epoch: 13/30.. Training loss: 0.329.. Test loss: 0.351.. Test Accuracy: 0

In [ ]:
# best results at epoch 22 and 25 based on accuracy